In [1]:
import numpy as np
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

def fetch_and_save_real_qpu_results(n=10, filename_prefix='real_qpu_bitarrays'):
    """
    Fetches the last `n` quantum jobs from IBM Cloud,
    extracts the bit_array (measurement data),
    and saves them to a timestamped .npy file.

    Args:
        n (int): Number of recent jobs to fetch.
        filename_prefix (str): Prefix for the output file name.
    """
    service = QiskitRuntimeService()
    jobs = service.jobs(limit=n)
    bitarrays = {}

    print(f"📡 Fetching last {len(jobs)} job(s)...")

    for i, job in enumerate(jobs):
        try:
            result = job.result()
            pub_result = result[0]
            bit_array = pub_result.data.meas
            job_id = job.job_id()
            bitarrays[f'job_{i}_{job_id}'] = bit_array
            print(f"✅ Collected job {i+1}/{n} → ID: {job_id}")
        except Exception as e:
            print(f"⚠️ Skipped job {i+1} → ID: {job.job_id()}: {e}")

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.npy"
    np.save(filename, bitarrays)
    print(f"\n💾 Saved all bitarrays to '{filename}'")


In [2]:
fetch_and_save_real_qpu_results(n=10)


📡 Fetching last 10 job(s)...
✅ Collected job 1/10 → ID: d1s512g1mggs73b7ht30
✅ Collected job 2/10 → ID: d1s3ubmjpjps738measg
✅ Collected job 3/10 → ID: d1s3t901mggs73b7gscg
✅ Collected job 4/10 → ID: d1s3sb5s0e1s73bmsiqg
✅ Collected job 5/10 → ID: d1s3r96jpjps738me7n0
✅ Collected job 6/10 → ID: d1s3r0ejpjps738me7f0
✅ Collected job 7/10 → ID: d1s3qoejpjps738me770
✅ Collected job 8/10 → ID: d1s3p26jpjps738me5jg
✅ Collected job 9/10 → ID: d1s3ojk7uv5s73a74i70
✅ Collected job 10/10 → ID: d1s3o5c7uv5s73a74hq0

💾 Saved all bitarrays to 'real_qpu_bitarrays_20250717_102235.npy'


In [3]:
fetch_and_save_real_qpu_results(n=5)


📡 Fetching last 5 job(s)...
✅ Collected job 1/5 → ID: d1s512g1mggs73b7ht30
✅ Collected job 2/5 → ID: d1s3ubmjpjps738measg
✅ Collected job 3/5 → ID: d1s3t901mggs73b7gscg
✅ Collected job 4/5 → ID: d1s3sb5s0e1s73bmsiqg
✅ Collected job 5/5 → ID: d1s3r96jpjps738me7n0

💾 Saved all bitarrays to 'real_qpu_bitarrays_20250717_102236.npy'


In [4]:
bitarrays = np.load('real_qpu_bitarrays_20250717_101329.npy', allow_pickle=True).item()

# Example use:
for job_id, bit_array in bitarrays.items():
    print(f"{job_id}: {bit_array}")


job_0_d1s512g1mggs73b7ht30: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_1_d1s3ubmjpjps738measg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_2_d1s3t901mggs73b7gscg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_3_d1s3sb5s0e1s73bmsiqg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_4_d1s3r96jpjps738me7n0: BitArray(<shape=(), num_shots=10000, num_bits=32>)


In [5]:
bitarrays = np.load('all_real_bitarrays.npy', allow_pickle=True).item()

# For example:
for job_id, bit_array in bitarrays.items():
    print(f"{job_id}: {bit_array}")


job_0_d1s512g1mggs73b7ht30: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_1_d1s3ubmjpjps738measg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_2_d1s3t901mggs73b7gscg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_3_d1s3sb5s0e1s73bmsiqg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_4_d1s3r96jpjps738me7n0: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_5_d1s3r0ejpjps738me7f0: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_6_d1s3qoejpjps738me770: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_7_d1s3p26jpjps738me5jg: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_8_d1s3ojk7uv5s73a74i70: BitArray(<shape=(), num_shots=10000, num_bits=32>)
job_9_d1s3o5c7uv5s73a74hq0: BitArray(<shape=(), num_shots=10000, num_bits=32>)


In [6]:
import numpy as np
import datetime
import csv
from qiskit_ibm_runtime import QiskitRuntimeService

def fetch_and_save_jobs_separately_with_summary(n=10, filename_prefix='bitarray_job', summary_file='bitarray_summary.csv'):
    """
    Fetches the last `n` IBM Quantum jobs, saves each bit_array in a separate .npy file,
    and creates a summary CSV with job info.
    """
    service = QiskitRuntimeService()
    jobs = service.jobs(limit=n)

    print(f"📡 Fetching and saving the last {len(jobs)} job(s)...\n")

    summary_data = []

    for i, job in enumerate(jobs):
        try:
            result = job.result()
            pub_result = result[0]
            bit_array = pub_result.data.meas

            job_id = job.job_id()
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{filename_prefix}_{i+1}_{job_id}_{timestamp}.npy"
            
            # Save the individual bit array
            np.save(filename, bit_array)
            print(f"✅ Saved Job {i+1}/{n} → '{filename}'")

            # Add summary entry
            summary_data.append({
                'Job Number': i + 1,
                'Job ID': job_id,
                'Filename': filename,
                'Timestamp': timestamp
            })

        except Exception as e:
            print(f"⚠️ Could not fetch job {i+1} (ID: {job.job_id()}): {e}")

    # Save CSV summary
    with open(summary_file, 'w', newline='') as csvfile:
        fieldnames = ['Job Number', 'Job ID', 'Filename', 'Timestamp']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(summary_data)

    print(f"\n🧾 Summary saved to '{summary_file}' with {len(summary_data)} entries.")


In [7]:
fetch_and_save_jobs_separately_with_summary(n=10)


📡 Fetching and saving the last 10 job(s)...

✅ Saved Job 1/10 → 'bitarray_job_1_d1s512g1mggs73b7ht30_20250717_102238.npy'
✅ Saved Job 2/10 → 'bitarray_job_2_d1s3ubmjpjps738measg_20250717_102238.npy'
✅ Saved Job 3/10 → 'bitarray_job_3_d1s3t901mggs73b7gscg_20250717_102238.npy'
✅ Saved Job 4/10 → 'bitarray_job_4_d1s3sb5s0e1s73bmsiqg_20250717_102238.npy'
✅ Saved Job 5/10 → 'bitarray_job_5_d1s3r96jpjps738me7n0_20250717_102239.npy'
✅ Saved Job 6/10 → 'bitarray_job_6_d1s3r0ejpjps738me7f0_20250717_102239.npy'
✅ Saved Job 7/10 → 'bitarray_job_7_d1s3qoejpjps738me770_20250717_102239.npy'
✅ Saved Job 8/10 → 'bitarray_job_8_d1s3p26jpjps738me5jg_20250717_102239.npy'
✅ Saved Job 9/10 → 'bitarray_job_9_d1s3ojk7uv5s73a74i70_20250717_102239.npy'
✅ Saved Job 10/10 → 'bitarray_job_10_d1s3o5c7uv5s73a74hq0_20250717_102239.npy'

🧾 Summary saved to 'bitarray_summary.csv' with 10 entries.


In [8]:
bit_array = pub_result.data.meas

# Convert BitArray to dictionary
bit_dict = dict(bit_array)

# Save the .npy file
np.save(f"{base_filename}.npy", bit_dict)

# Plot the histogram
labels = list(bit_dict.keys())
values = list(bit_dict.values())

plt.figure(figsize=(10, 5))
plt.bar(labels, values, color='skyblue')
plt.xticks(rotation=90)
plt.xlabel("Bitstrings")
plt.ylabel("Counts")
plt.title(f"Job {i+1} – Bitstring Distribution")
plt.tight_layout()
plt.savefig(f"{base_filename}.png")
plt.close()


NameError: name 'pub_result' is not defined

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

def fetch_and_save_jobs_separately(n=5):
    jobs = service.jobs(limit=n)
    for i, job in enumerate(jobs):
        try:
            job_id = job.job_id()
            print(f"\n📦 Fetching Job {i+1} (ID: {job_id})...")
            
            # Get result
            result = job.result()
            primitive_result = result[0]
            bit_array = primitive_result.data.meas
            bit_dict = dict(bit_array)

            print(f"✅ Job {i+1} fetched. Top 5 bitstrings:")
            for k, v in list(bit_dict.items())[:5]:
                print(f"{k}: {v}")
            
            # Save .npy
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename_base = f"bitarray_job_{i+1}_{job_id}_{timestamp}"
            np.save(f"{filename_base}.npy", bit_dict)

            # Plot
            plt.figure(figsize=(10, 4))
            plt.bar(bit_dict.keys(), bit_dict.values(), color='skyblue')
            plt.xticks(rotation=90)
            plt.xlabel("Bitstrings")
            plt.ylabel("Counts")
            plt.title(f"Job {i+1} Bitstring Distribution")
            plt.tight_layout()
            plt.savefig(f"{filename_base}.png")
            plt.show()
            plt.close()

        except Exception as e:
            print(f"⚠️ Failed to process job {i+1} (ID: {job_id}): {e}")


In [11]:
fetch_and_save_jobs_separately(n=5)



📦 Fetching Job 1 (ID: d1s512g1mggs73b7ht30)...
⚠️ Failed to process job 1 (ID: d1s512g1mggs73b7ht30): The input array must have at least two axes.

📦 Fetching Job 2 (ID: d1s3ubmjpjps738measg)...
⚠️ Failed to process job 2 (ID: d1s3ubmjpjps738measg): The input array must have at least two axes.

📦 Fetching Job 3 (ID: d1s3t901mggs73b7gscg)...
⚠️ Failed to process job 3 (ID: d1s3t901mggs73b7gscg): The input array must have at least two axes.

📦 Fetching Job 4 (ID: d1s3sb5s0e1s73bmsiqg)...
⚠️ Failed to process job 4 (ID: d1s3sb5s0e1s73bmsiqg): The input array must have at least two axes.

📦 Fetching Job 5 (ID: d1s3r96jpjps738me7n0)...
⚠️ Failed to process job 5 (ID: d1s3r96jpjps738me7n0): The input array must have at least two axes.


In [12]:
jobs = service.jobs(limit=5)

for i, job in enumerate(jobs):
    try:
        job_id = job.job_id()
        print(f"\n📦 Job {i+1} (ID: {job_id})")

        result = job.result()
        primitive_result = result[0]

        meas_data = primitive_result.data.meas
        print(f"🔍 Type of meas: {type(meas_data)}")
        print(f"🔢 Sample data (first 10 rows):\n{meas_data[:10]}")
        
    except Exception as e:
        print(f"⚠️ Failed to process job {i+1}: {e}")



📦 Job 1 (ID: d1s512g1mggs73b7ht30)
🔍 Type of meas: <class 'qiskit.primitives.containers.bit_array.BitArray'>
🔢 Sample data (first 10 rows):
BitArray(<shape=(), num_shots=10, num_bits=32>)

📦 Job 2 (ID: d1s3ubmjpjps738measg)
🔍 Type of meas: <class 'qiskit.primitives.containers.bit_array.BitArray'>
🔢 Sample data (first 10 rows):
BitArray(<shape=(), num_shots=10, num_bits=32>)

📦 Job 3 (ID: d1s3t901mggs73b7gscg)
🔍 Type of meas: <class 'qiskit.primitives.containers.bit_array.BitArray'>
🔢 Sample data (first 10 rows):
BitArray(<shape=(), num_shots=10, num_bits=32>)

📦 Job 4 (ID: d1s3sb5s0e1s73bmsiqg)
🔍 Type of meas: <class 'qiskit.primitives.containers.bit_array.BitArray'>
🔢 Sample data (first 10 rows):
BitArray(<shape=(), num_shots=10, num_bits=32>)

📦 Job 5 (ID: d1s3r96jpjps738me7n0)
🔍 Type of meas: <class 'qiskit.primitives.containers.bit_array.BitArray'>
🔢 Sample data (first 10 rows):
BitArray(<shape=(), num_shots=10, num_bits=32>)


In [13]:
from qiskit.result import marginal_counts
from qiskit.visualization import plot_histogram

for i, job in enumerate(jobs):
    try:
        print(f"\n📦 Job {i+1} (ID: {job.job_id()})")

        result = job.result()
        pub_result = result[0]
        bit_array = pub_result.data.meas

        # ✅ Convert BitArray to counts dictionary
        counts = bit_array.to_counts()

        # ✅ Print a few entries
        print("Sample counts:", dict(list(counts.items())[:5]))

        # ✅ Plot histogram
        plot_histogram(counts).show()

        # (Optional) Save counts if needed
        # with open(f"job_{i+1}_counts.json", "w") as f:
        #     json.dump(counts, f)

    except Exception as e:
        print(f"⚠️ Failed to process job {i+1}: {e}")



📦 Job 1 (ID: d1s512g1mggs73b7ht30)
⚠️ Failed to process job 1: 'BitArray' object has no attribute 'to_counts'

📦 Job 2 (ID: d1s3ubmjpjps738measg)
⚠️ Failed to process job 2: 'BitArray' object has no attribute 'to_counts'

📦 Job 3 (ID: d1s3t901mggs73b7gscg)
⚠️ Failed to process job 3: 'BitArray' object has no attribute 'to_counts'

📦 Job 4 (ID: d1s3sb5s0e1s73bmsiqg)
⚠️ Failed to process job 4: 'BitArray' object has no attribute 'to_counts'

📦 Job 5 (ID: d1s3r96jpjps738me7n0)
⚠️ Failed to process job 5: 'BitArray' object has no attribute 'to_counts'


In [14]:
from collections import Counter
from qiskit.visualization import plot_histogram

for i, job in enumerate(jobs):
    try:
        print(f"\n📦 Job {i+1} (ID: {job.job_id()})")

        result = job.result()
        pub_result = result[0]
        bit_array = pub_result.data.meas  # BitArray object

        # ✅ Convert each shot to bitstring (e.g., '0101')
        bitstrings = [''.join(map(str, bits)) for bits in bit_array]

        # ✅ Count occurrences of each bitstring
        counts = dict(Counter(bitstrings))

        print("Sample counts:", dict(list(counts.items())[:5]))

        # ✅ Plot histogram (may not show in Qbraid — let me know)
        plot_histogram(counts).show()

    except Exception as e:
        print(f"⚠️ Failed to process job {i+1}: {e}")



📦 Job 1 (ID: d1s512g1mggs73b7ht30)
⚠️ Failed to process job 1: The input array must have at least two axes.

📦 Job 2 (ID: d1s3ubmjpjps738measg)
⚠️ Failed to process job 2: The input array must have at least two axes.

📦 Job 3 (ID: d1s3t901mggs73b7gscg)
⚠️ Failed to process job 3: The input array must have at least two axes.

📦 Job 4 (ID: d1s3sb5s0e1s73bmsiqg)
⚠️ Failed to process job 4: The input array must have at least two axes.

📦 Job 5 (ID: d1s3r96jpjps738me7n0)
⚠️ Failed to process job 5: The input array must have at least two axes.


In [15]:
from collections import Counter
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

for i, job in enumerate(jobs):
    try:
        print(f"\n📦 Job {i+1} (ID: {job.job_id()})")

        result = job.result()
        pub_result = result[0]
        bit_array = pub_result.data.meas  # BitArray object

        # 👉 Ensure array is at least 2D
        array_data = bit_array.to_numpy()
        if array_data.ndim == 1:
            array_data = array_data.reshape(1, -1)  # Make it (1, n_bits)

        # ✅ Convert each shot to bitstring
        bitstrings = [''.join(map(str, row)) for row in array_data]

        # ✅ Count bitstring occurrences
        counts = dict(Counter(bitstrings))

        print("Sample counts:", dict(list(counts.items())[:5]))

        # ✅ Plot histogram
        fig = plot_histogram(counts)
        fig.show()

    except Exception as e:
        print(f"⚠️ Failed to process job {i+1}: {e}")



📦 Job 1 (ID: d1s512g1mggs73b7ht30)
⚠️ Failed to process job 1: 'BitArray' object has no attribute 'to_numpy'

📦 Job 2 (ID: d1s3ubmjpjps738measg)
⚠️ Failed to process job 2: 'BitArray' object has no attribute 'to_numpy'

📦 Job 3 (ID: d1s3t901mggs73b7gscg)
⚠️ Failed to process job 3: 'BitArray' object has no attribute 'to_numpy'

📦 Job 4 (ID: d1s3sb5s0e1s73bmsiqg)
⚠️ Failed to process job 4: 'BitArray' object has no attribute 'to_numpy'

📦 Job 5 (ID: d1s3r96jpjps738me7n0)
⚠️ Failed to process job 5: 'BitArray' object has no attribute 'to_numpy'


In [16]:
from collections import Counter
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

for i, job in enumerate(jobs):
    try:
        print(f"\n📦 Job {i+1} (ID: {job.job_id()})")

        result = job.result()
        pub_result = result[0]
        bit_array = pub_result.data.meas  # BitArray

        # ✅ Convert BitArray to list of bitstrings
        bitstrings = bit_array.to_labels()  # returns ['0101', '1110', ...]

        # ✅ Count occurrences
        counts = dict(Counter(bitstrings))
        print("Sample counts:", dict(list(counts.items())[:5]))

        # ✅ Plot histogram
        fig = plot_histogram(counts)
        fig.show()

    except Exception as e:
        print(f"⚠️ Failed to process job {i+1}: {e}")



📦 Job 1 (ID: d1s512g1mggs73b7ht30)
⚠️ Failed to process job 1: 'BitArray' object has no attribute 'to_labels'

📦 Job 2 (ID: d1s3ubmjpjps738measg)
⚠️ Failed to process job 2: 'BitArray' object has no attribute 'to_labels'

📦 Job 3 (ID: d1s3t901mggs73b7gscg)
⚠️ Failed to process job 3: 'BitArray' object has no attribute 'to_labels'

📦 Job 4 (ID: d1s3sb5s0e1s73bmsiqg)
⚠️ Failed to process job 4: 'BitArray' object has no attribute 'to_labels'

📦 Job 5 (ID: d1s3r96jpjps738me7n0)
⚠️ Failed to process job 5: 'BitArray' object has no attribute 'to_labels'


In [17]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Load your IBM credentials (if not already loaded)
service = QiskitRuntimeService()

# Fetch the job
job = service.job(job_id='d1s512g1mggs73b7ht30')


In [18]:
result = job.result()


In [19]:
print(type(result))  # Typically <class 'qiskit.primitives.primitive_job.PrimitiveJobResult'>
print(result)        # Might show summary


<class 'qiskit.primitives.containers.primitive_result.PrimitiveResult'>
PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=10000, num_bits=32>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-07-17 01:23:24', stop='2025-07-17 01:23:28', size=10000>)])}, 'version': 2})


In [20]:
# Try these one by one:
print(dir(result))  # See available fields like .data or .results

# Try this if it exists:
if hasattr(result, 'results'):
    print(result.results)


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', '_metadata', '_pub_results', 'metadata']


In [21]:
# Try these one by one:
print(dir(result))  # See available fields like .data or .results

# Try this if it exists:
if hasattr(result, 'results'):
    print(result.results)


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', '_metadata', '_pub_results', 'metadata']


In [22]:
meas = result[0].data.meas  # or possibly result.data.meas
print(meas)
print(type(meas))
print("Shape:", getattr(meas, "shape", "No shape"))
print("Shots:", getattr(meas, "num_shots", "No info"))
print("Bits :", getattr(meas, "num_bits", "No info"))


BitArray(<shape=(), num_shots=10000, num_bits=32>)
<class 'qiskit.primitives.containers.bit_array.BitArray'>
Shape: ()
Shots: 10000
Bits : 32


In [23]:
# If you see valid shots/bits
try:
    print("First 5 bitstrings:", meas[:5])
except Exception as e:
    print("Error showing bits:", e)

# Try converting
try:
    print("Counts:", meas.to_counts())
except Exception as e:
    print("to_counts() failed:", e)

try:
    print("As NumPy array:", meas.to_numpy()[:5])
except Exception as e:
    print("to_numpy() failed:", e)


First 5 bitstrings: BitArray(<shape=(), num_shots=5, num_bits=32>)
to_counts() failed: 'BitArray' object has no attribute 'to_counts'
to_numpy() failed: 'BitArray' object has no attribute 'to_numpy'


In [24]:
import json

with open('job_d1s512g1mggs73b7ht30_raw.json', 'w') as f:
    json.dump(job.to_dict(), f, indent=2, default=str)


AttributeError: 'RuntimeJobV2' object has no attribute 'to_dict'

In [25]:
# Access metadata
print(result.metadata)


{'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-07-17 01:23:24', stop='2025-07-17 01:23:28', size=10000>)])}, 'version': 2}


In [26]:
print(type(result1))
print(dir(result1))
print(result1)


NameError: name 'result1' is not defined

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

def fetch_and_save_jobs_separately(n=5):
    jobs = service.jobs(limit=n)
    for i, job in enumerate(jobs):
        try:
            job_id = job.job_id()
            print(f"\n📦 Fetching Job {i+1} (ID: {job_id})...")
            
            # Get result
            result = job.result()
            primitive_result = result[0]
            bit_array = primitive_result.data.meas
            bit_dict = dict(bit_array)

            print(f"✅ Job {i+1} fetched. Top 5 bitstrings:")
            for k, v in list(bit_dict.items())[:5]:
                print(f"{k}: {v}")
            
            # Save .npy
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename_base = f"bitarray_job_{i+1}_{job_id}_{timestamp}"
            np.save(f"{filename_base}.npy", bit_dict)

            # Plot
            plt.figure(figsize=(10, 4))
            plt.bar(bit_dict.keys(), bit_dict.values(), color='skyblue')
            plt.xticks(rotation=90)
            plt.xlabel("Bitstrings")
            plt.ylabel("Counts")
            plt.title(f"Job {i+1} Bitstring Distribution")
            plt.tight_layout()
            plt.savefig(f"{filename_base}.png")
            plt.show()
            plt.close()

        except Exception as e:
            print(f"⚠️ Failed to process job {i+1} (ID: {job_id}): {e}")


In [28]:
extract_and_plot_bitarray_data(result1)


NameError: name 'extract_and_plot_bitarray_data' is not defined

In [29]:
print(type(obj))
print(dir(obj))


NameError: name 'obj' is not defined